In [1]:
!pip install exifread

# Extracting GPS Information from Images

(You will need to modify this script based on how your dataset is stored in order to execute the code.).


In [ ]:
import os

# train_or_test_or_validation could be either train, test, or validation.
train_or_test_or_validation = "train" # it could also be test or validation
PATH_TO_YOUR_DATA_FOLDER = "~/Desktop/CIS519"
directory_path = f"{PATH_TO_YOUR_DATA_FOLDER}/{train_or_test_or_validation}"
output_csv = "metadata.csv"
output_csv = os.path.join(directory_path, output_csv)

In [3]:
import exifread, csv

def get_exif_data(image_path):
    with open(image_path, 'rb') as image_file:
        tags = exifread.process_file(image_file)
    return tags

def export_exif_to_json(exif_data, output_file):
    # Convert tags to a serializable format
    exif_data_serializable = {str(tag): str(value) for tag, value in exif_data.items()}
    with open(output_file, 'w') as json_file:
        json.dump(exif_data_serializable, json_file, indent=4)

In [4]:
# Function to convert GPS coordinates in degrees, minutes, and seconds to decimal degrees
def convert_to_decimal_degrees(value):
    d, m, s = value.values
    return d.num / d.den + (m.num / m.den) / 60 + (s.num / s.den) / 3600

### You will need to create subfolders in {PATH_TO_YOUR_DATA_FOLDER} for each split (train/test/validation) or just (train/test). Next, place the corresponding images into each split after randomly shuffling them. Then, create a metadata.csv file for each split and place it in the corresponding directory. Note that the current code only works for jpeg images. If the exported images are in some other format, convert them to .jpg before running this code.

In [ ]:
with open(output_csv, mode='w', newline='') as csv_file:
    fieldnames = ['file_name', 'Latitude', 'Longitude']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write the header row
    writer.writeheader()
    for filename in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, filename)):
            exif_data = get_exif_data(os.path.join(directory_path, filename))
            if exif_data:
                gps_latitude = exif_data.get('GPS GPSLatitude', None)
                gps_latitude_ref = exif_data.get('GPS GPSLatitudeRef', None)
                gps_longitude = exif_data.get('GPS GPSLongitude', None)
                gps_longitude_ref = exif_data.get('GPS GPSLongitudeRef', None)
                if gps_latitude and gps_longitude:
                    # Convert latitude and longitude to decimal degrees
                    latitude = convert_to_decimal_degrees(gps_latitude)
                    longitude = convert_to_decimal_degrees(gps_longitude)

                    # Adjust for N/S and E/W reference
                    if gps_latitude_ref.values[0] == 'S':
                        latitude = -latitude
                    if gps_longitude_ref.values[0] == 'W':
                        longitude = -longitude

                    # Write the data to the CSV file
                    writer.writerow({'file_name': filename, 'Latitude': latitude, 'Longitude': longitude})

# Uploading and Reading a Dataset on Hugging Face

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### You need to ensure that your Hugging Face token has both read and write access to your repository and Hugging Face organization.

In [ ]:
!pip install huggingface_hub

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir=PATH_TO_YOUR_DATA_FOLDER)

dataset.push_to_hub("{YOUR ORGANIZATION NAME}/{DATASET NAME}", private=True)

In [ ]:
### After uploading, you can access your data using this code
dataset_test = load_dataset("{YOUR ORGANIZATION NAME}/{DATASET NAME}", split="{YOUR SPLIT}")